In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas as pd
from datetime import datetime

autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

nome = 'CREC'
indisponivel = 'indisponivel'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    
    # inserindo usuário
    nav.find_element(By.XPATH, '//*[@id="id_username"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element(By.XPATH, '//*[@id="id_password"]').click()
    nav.find_element(By.XPATH, '//*[@id="id_password"]').send_keys(senha)

    nav.find_element(By.XPATH, '//*[@id="ok"]').click()

In [ ]:
# extraindo para lista
def table_to_list():
    tabela = []

    lista = nav.find_element(By.TAG_NAME, 'table')
    td = lista.find_elements(By.TAG_NAME, 'td')

    for i in td:
        tabela.append(i.text)

    return tabela

In [ ]:
def build_df(lista):
    df_tabela = pd.DataFrame()


    tombos = []
    ele_des = []
    descricao = []
    cargas = []
    c_contabil = []
    salas = []
    dt_entrada = []
    fornecedores = []
    valores = []

    #  remove o primeiro elemento ta tabela, informação quebrada    
    for excluir in range(0, len(lista), 1201):
        lista.pop(excluir)

    for tombo in range(1, len(lista)+1, 12):
        if lista[tombo] == '':
            lista.pop(tombo)

    # listas para montar o data frame
    for tombo in range(1, len(lista), 12):
        tombos.append(lista[tombo])

    for ed in range(2, len(lista), 12):
        ele_des.append(lista[ed])

    for des in range(3, len(lista), 12):
        descricao.append(lista[des])

    for carga in range(4, len(lista), 12):
        cargas.append(lista[carga])

    for c in range(5, len(lista), 12):
        c_contabil.append(lista[c])

    for sala in range(6, len(lista), 12):
        salas.append(lista[sala])

    for data in range(8, len(lista), 12):
        dt_entrada.append(lista[data])
    
    for f in range(10, len(lista), 12):
        fornecedores.append(lista[f])

    for valor in range(11, len(lista), 12):
        valores.append(lista[valor])



    #  montagem do data frame
    df_tabela['tombo'] = tombos
    df_tabela['tombo'] = df_tabela['tombo'].astype(str).str[:-6]

    df_tabela['ed'] = ele_des

    df_tabela['descricao'] = descricao

    df_tabela['carga'] = cargas
    df_tabela['sala'] = salas

    df_tabela['c_contabil'] = c_contabil

    df_tabela['dt_entrada'] = dt_entrada
#     df_tabela['dt_entrada'] = pd.to_datetime(df_tabela['dt_entrada'], infer_datetime_format=True)

    df_tabela['fornecedor'] = fornecedores
    
    df_tabela['valor'] = valores
    df_tabela['valor'] = df_tabela['valor'].str.replace('.','',regex=True)
    df_tabela['valor'] = df_tabela['valor'].str.replace(',','.',regex=True)
    df_tabela['valor'] = pd.to_numeric(df_tabela['valor'])
    df_tabela['valor'] = df_tabela['valor'].round(2)

    return df_tabela

In [ ]:
def pagination():
    
    abrir_suap()
    print(f'Iniciou as: {datetime.now()}')
    pag = 1
    campi = 12
    saida = False
    complete_df = pd.DataFrame()
    
    
    while saida is False:
# cafi 12 - crec 4 - cbja - 7
        try:
            nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
            celula = nav.find_element(By.CLASS_NAME, 'row1')

            if celula:
                suap_list = table_to_list()
                suap_df = build_df(suap_list)
                complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                pag += 1

            else:
                saida = True

        except NoSuchElementException:
            
            try:
                error = nav.find_element(By.ID, 'main-frame-error')
                print(f'Caiu as: {datetime.now()} - na pagina {pag}')

                if error:

                    nav.quit()
                    abrir_suap()
                    print(f'Retomou as: {datetime.now()} na pagina {pag}')
                    nav.get(f'https://suap.ifpe.edu.br/patrimonio/inventario_busca/?page={pag}&status=&empenho=&empenho=&data_final=&sala=&numero_serie=&numero=&rotulos=&data_inicial=&estado_conservacao=6&elemento_despesa=&valor=&responsavel=&responsavel=&contabil=&fornecedor=&fornecedor=&setor_responsavel=&campus={campi}&descricao=')
                    celula = nav.find_element(By.CLASS_NAME, 'row1')

                    if celula:
                        pag += 1
                        suap_list = table_to_list()
                        suap_df = build_df(suap_list)
                        complete_df = pd.concat([complete_df, suap_df], ignore_index= True)
                        print(f'pagina pos erro - {pag}')
                        continue

                    else:
                        break
                        
            except NoSuchElementException:
                print('acabaram as paginas')
                break
            
                

    print(f'Terminou as: {datetime.now()} na pagina {pag}')
    display(complete_df)
    complete_df.to_excel(f'extracoes_suap/{data_e_hora}_{nome}_relatorio_patrimonio.xlsx', index=False)
    
#     nav.quit()
    

In [ ]:
def complement_item(item_list):
    
    abrir_suap()
    
    empenho = []
    ug_ne = []
    uasg = []
    nf = []
    dt_nf = []
    type_entry = []
    supplier = []
    process = []
    
    for linha in item_list:
        
#         print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")
            nav.find_element(By.LINK_TEXT, "Detalhes da Entrada").click()

            ne = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
            ug = ne[18:22]
            ne = ne[:12]
            empenho.append(ne)
            ug_ne.append(ug)
            # print(empenho)
        
            gest = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
            uasg.append(gest)
            # print(uasg)

            note = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
            nf.append(note)
            # print(nf)

            data = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
            dt_nf.append(data)
            # print(data)
            
            entry = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[3]/td").get_attribute('innerText')
            type_entry.append(entry)
            
            sp = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td/a").get_attribute('innerText')
            supplier.append(sp)
            
            pc = nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/table[1]/tbody/tr[13]/td").get_attribute('innerText')
            process.append(pc)


            
        except (NoSuchElementException, TimeoutException):
            # print(indisponivel)
            empenho.append(indisponivel)
            ug_ne.append(indisponivel)
            uasg.append(indisponivel)
            nf.append(indisponivel)
            dt_nf.append(indisponivel)
            type_entry.append(indisponivel)
            supplier.append(indisponivel)
            process.append(indisponivel)            
            continue    
        
    table = pd.DataFrame()
    table['empenho'] = empenho
    table['ug_empenho'] = ug_ne
    table['uasg'] = uasg
    table['nt_fiscal'] = nf
    table['dt_nf'] = dt_nf
    table['tipo_entrada'] = type_entry
    table['fornecedor'] = supplier
    table['processo'] = process
    
#     display(tabela_impressao)
    
    table.to_excel(f'{data_e_hora}_{nome}.xlsx', index=False, sheet_name="plan1")
        

#     nav.quit()

In [ ]:
cafi = pd.read_excel('extracoes_suap/2022-11-13_16-14_CAFI_relatorio_patrimonio.xlsx')
complement_item(cafi['tombo'])